# 1. 引入包

In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier

# 2. 读取数据

In [40]:
# 读取sel日志，排序
sel_data = pd.read_csv('../dataset/raw_dataset/additional_sel_log_dataset.csv')
sel_data.sort_values(by=['sn', 'time'], inplace=True)
sel_data.reset_index(drop=True, inplace=True)
sel_data

,sn,time,msg
0,SERVER_30000,2020-10-13 03:06:01,Power Supply Power Unit | Fully Redundant
1,SERVER_30000,2020-10-13 03:06:11,Processor CPU 1 | Configuration Error | Asserted
2,SERVER_30001,2020-07-31 17:24:10,System ACPI Power State ACPI_PWR | S4/S5: sof...
3,SERVER_30001,2020-07-31 17:24:22,System ACPI Power State ACPI_PWR | S0/G0: wor...
4,SERVER_30001,2020-08-11 16:31:36,System ACPI Power State ACPI_PWR | S4/S5: sof...
...,...,...,...
732669,SERVER_42314,2019-11-15 13:37:34,Power Supply PS2_Status | Power Supply AC los...
732670,SERVER_42314,2019-11-15 13:37:36,Power Supply PS2_Status | Power Supply AC los...
732671,SERVER_42314,2020-02-03 16:06:16,Power Supply PS1_Status | Presence detected |...
732672,SERVER_42314,2020-02-03 16:06:16,Power Supply PS2_Status | Presence detected |...


# 3. 分词

In [3]:
# 取出每台服务器的最后十条日志
# sn_list = sel_data['sn'].drop_duplicates(keep='first').to_list()
# tail_msg_list = ['.'.join(sel_data[sel_data['sn']==i]['msg'].tail(10).to_list()) for i in sn_list]
# tokenized_sent = [word_tokenize(s.lower()) for s in tail_msg_list]

In [41]:
# 这一个的速度会更快一些
sn_list = sel_data['sn'].drop_duplicates(keep='first').to_list()
sel_data_x = sel_data.groupby('sn').tail(10)
sel_data_x['new'] = sel_data_x.groupby(['sn'])['msg'].transform(lambda x : '.'.join(x)) 
sel_data_x.drop_duplicates(subset=['sn'],keep='first',inplace=True)
sel_data_x = sel_data_x.reset_index(drop=True)

tail_msg_list_y = sel_data_x['new'].tolist()
tokenized_sent = [word_tokenize(s.lower()) for s in tail_msg_list_y]

D:\ProgramData\Anaconda3\envs\soar_new\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ProgramData\Anaconda3\envs\soar_new\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [45]:
tokenized_sent

[['power',
  'supply',
  'power',
  'unit',
  '|',
  'fully',
  'redundant',
  '.',
  'processor',
  'cpu',
  '1',
  '|',
  'configuration',
  'error',
  '|',
  'asserted'],
 ['power',
  'supply',
  'psu0_status',
  '|',
  'presence',
  'detected',
  '|',
  'asserted',
  '.',
  'power',
  'supply',
  'psu1_status',
  '|',
  'presence',
  'detected',
  '|',
  'asserted',
  '.',
  'processor',
  'cpu0_status',
  '|',
  'presence',
  'detected',
  '|',
  'asserted',
  '.',
  'system',
  'boot',
  'initiated',
  'bios_boot_up',
  '|',
  'state',
  'asserted',
  '|',
  'asserted',
  '.',
  'system',
  'boot',
  'initiated',
  'bios_boot_up',
  '|',
  'state',
  'asserted',
  '|',
  'asserted',
  '.',
  'system',
  'boot',
  'initiated',
  'bios_boot_up',
  '|',
  'state',
  'asserted',
  '|',
  'asserted',
  '.',
  'system',
  'boot',
  'initiated',
  'bios_boot_up',
  '|',
  'state',
  'asserted',
  '|',
  'asserted',
  '.',
  'system',
  'boot',
  'initiated',
  'bios_boot_up',
  '|',
  '

In [46]:
tokenized_sent[0]

['power',
 'supply',
 'power',
 'unit',
 '|',
 'fully',
 'redundant',
 '.',
 'processor',
 'cpu',
 '1',
 '|',
 'configuration',
 'error',
 '|',
 'asserted']

In [43]:
len(sn_list)

11649

# 4. 训练Embedding模型（Doc2Vec）

In [64]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
model_doc = Doc2Vec(tagged_data, vector_size = 10, window = 5, min_count = 1, epochs = 100)

2022-03-11 13:52:37,293 : INFO : collecting all words and their counts
2022-03-11 13:52:37,294 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-03-11 13:52:37,392 : INFO : PROGRESS: at example #10000, processed 967385 words (9925134/s), 2355 word types, 0 tags
2022-03-11 13:52:37,407 : INFO : collected 2554 word types and 11649 unique tags from a corpus of 11649 examples and 1113904 words
2022-03-11 13:52:37,408 : INFO : Creating a fresh vocabulary
2022-03-11 13:52:37,418 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 2554 unique words (100.0%% of original 2554, drops 0)', 'datetime': '2022-03-11T13:52:37.418373', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2022-03-11 13:52:37,418 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 1113904 word corpus (100.0%% of original 1113904, drops

# 5. 构建树模型的训练集

In [91]:
# tqdm是用来显示进度条的
from tqdm import tqdm

label = pd.read_csv('../dataset/raw_dataset/preliminary_train_label_dataset.csv')
label.sort_values(by=['sn', 'fault_time'], inplace=True)
label.reset_index(drop=True, inplace=True)
train_data = []
for i, row in tqdm(label.iterrows(), total = len(label)):
    train_data.append(
        model_doc.infer_vector(
            word_tokenize(
                '.'.join(
                    sel_data[ (sel_data['sn']==row['sn'])&(sel_data['time']<=row['fault_time']) ].tail(10)['msg']
                    ).lower()
                )
            )
        )
train_feature = np.array(train_data)
train_label = label['label'].values

100%|██████████| 12260/12260 [14:31<00:00, 14.07it/s]


In [6]:
#  测试函数推倒式

# label = pd.read_csv('../dataset/raw_dataset/preliminary_train_label_dataset.csv')
# label.sort_values(by=['sn', 'fault_time'], inplace=True)
# label.reset_index(drop=True, inplace=True)
# train_data = []
# test = None
# for i, row in label.iterrows():
#     test = model.infer_vector(
#             word_tokenize(
#                 '.'.join(
#                     sel_data[ (sel_data['sn']==row['sn'])&(sel_data['time']<=row['fault_time']) ].tail(10)['msg']
#                     ).lower()
#                 )
#             )
#     break
# test

# 分割数据集

In [81]:
from keras.layers import Dense, Dropout
import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical

# train_label_to_cate = to_categorical(train_label, num_classes = 4)
# train_label_to_cate.shape

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_feature, train_label, test_size = 0.25)

In [82]:
# 查看划分后的数据集尺寸大小
print("训练集：", X_train.shape)
print("训练集标签：", y_train.shape)

训练集： (9195, 10)
训练集标签： (9195,)


# 6. 深度学习模型

## 6.2 多层感知机模型

In [83]:
def build_mlp(max_features):
    model = Sequential()
    model.add(Dense(units = 30, input_dim = max_features, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(units = 30, activation = "relu"))
    model.add(Dense(units = 4, activation = "softmax"))
    # sgd = SGD(lr = 0.002)

    model.compile(optimizer="adam", loss=tf.losses.sparse_categorical_crossentropy, metrics = ["accuracy"])
    return model
# 设置模型提前停止训练
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = build_mlp(10)
model.fit(X_train, y_train, epochs=100,  batch_size=128, verbose=1, callbacks = [callback])
score1 = model.evaluate(X_test, y_test)
score1

Epoch 1/100
72/72 [==============================] - 0s 3ms/step - loss: 1.2750 - accuracy: 0.5489
Epoch 2/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1488 - accuracy: 0.5619
Epoch 3/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1466 - accuracy: 0.5619
Epoch 4/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1470 - accuracy: 0.5619
Epoch 5/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1465 - accuracy: 0.5619
Epoch 6/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1460 - accuracy: 0.5619
Epoch 7/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1473 - accuracy: 0.5619
Epoch 8/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1455 - accuracy: 0.5619
Epoch 9/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1453 - accuracy: 0.5619
Epoch 10/100
72/72 [==============================] - 0s 3ms/step - loss: 1.1465 - accuracy: 0.5619
Epoch 11/

[1.1388195753097534, 0.563784658908844]

## 6.3 BiGRU 模型


In [68]:
# 获取映射的词汇量大小
len(model_doc.wv.key_to_index)

2554

In [84]:
from keras.layers.embeddings import Embedding
from keras.layers import GRU, Bidirectional, BatchNormalization,Flatten,LayerNormalization

def build_BiGRU(max_features, maxlen):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length = maxlen))
    model.add(LayerNormalization())
    model.add(Bidirectional(GRU(128)))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation = "softmax"))
    # model.add(Activation('softmax'))

    model.compile(optimizer="adam", loss=tf.losses.sparse_categorical_crossentropy, metrics = ["accuracy"])
    return model

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = build_BiGRU(len(model_doc.wv.key_to_index), 10)
model.fit(X_train, y_train, epochs=100,  batch_size=128, verbose=1, callbacks = [callback])
score1 = model.evaluate(X_test, y_test)
score1

Epoch 1/100
72/72 [==============================] - 3s 7ms/step - loss: 1.2033 - accuracy: 0.5476
Epoch 2/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1732 - accuracy: 0.5612
Epoch 3/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1572 - accuracy: 0.5619
Epoch 4/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1585 - accuracy: 0.5619
Epoch 5/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1545 - accuracy: 0.5619
Epoch 6/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1552 - accuracy: 0.5619
Epoch 7/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1563 - accuracy: 0.5619
Epoch 8/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1549 - accuracy: 0.5619
Epoch 9/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1512 - accuracy: 0.5619
Epoch 10/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1503 - accuracy: 0.5619
Epoch 11/

[1.1398656368255615, 0.563784658908844]

## 6.4 conv1D

In [85]:
from keras.layers import Conv1D
def build_conv1(max_features, maxlen):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length = maxlen))
    model.add(Conv1D(1, 3, activation = tf.nn.relu))
    model.add(LayerNormalization())
    model.add(Conv1D(1, 8, activation = tf.nn.relu))
    model.add(LayerNormalization())
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(4, activation = "softmax"))
    # model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='rmsprop', metrics = ["accuracy"])
    model.compile(optimizer="adam", loss=tf.losses.sparse_categorical_crossentropy, metrics = ["accuracy"])
    return model

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = build_BiGRU(len(model_doc.wv.key_to_index), 10)
model.fit(X_train, y_train, epochs=100,  batch_size=128, verbose=1, callbacks = [callback])
score1 = model.evaluate(X_test, y_test)
score1

Epoch 1/100
72/72 [==============================] - 3s 7ms/step - loss: 1.1872 - accuracy: 0.5513
Epoch 2/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1675 - accuracy: 0.5612
Epoch 3/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1552 - accuracy: 0.5619
Epoch 4/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1587 - accuracy: 0.5618
Epoch 5/100
72/72 [==============================] - 1s 7ms/step - loss: 1.1654 - accuracy: 0.5619
Epoch 6/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1533 - accuracy: 0.5619
Epoch 7/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1520 - accuracy: 0.5619
Epoch 8/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1534 - accuracy: 0.5619
Epoch 9/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1528 - accuracy: 0.5619
Epoch 10/100
72/72 [==============================] - 1s 8ms/step - loss: 1.1523 - accuracy: 0.5619
Epoch 11/

[1.1383649110794067, 0.563784658908844]

# 7. 机器学习模型

## 7.1 随机森林

In [86]:
rf = RandomForestClassifier(oob_score=True)
rf.fit(X_train, y_train)
score = rf.score(X_test, y_test)
score

0.5611745513866232

## 7.2 梯度提升树

In [87]:
from sklearn.ensemble import GradientBoostingClassifier as clf_gbo

clf = clf_gbo(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=2021, verbose= 1).fit(X_train, y_train)
score = clf.score(X_test, y_test)
score

      Iter       Train Loss   Remaining Time 
         1           1.1428            3.96s
         2           1.1410            3.97s
         3           1.1397            3.91s
         4           1.1383            3.82s
         5           1.1372            3.76s
         6           1.1360            3.74s
         7           1.1350            3.75s
         8           1.1340            3.69s
         9           1.1329            3.64s
        10           1.1320            3.58s
        20           1.1248            3.16s
        30           1.1184            2.77s
        40           1.1131            2.36s
        50           1.1087            1.96s
        60           1.1048            1.57s
        70           1.1012            1.18s
        80           1.0979            0.78s
        90           1.0948            0.39s
       100           1.0921            0.00s


0.5605220228384992

In [ ]:
# submit = pd.read_csv('../dataset/raw_dataset/preliminary_submit_dataset.csv')
# submit.sort_values(by=['sn', 'fault_time'], inplace=True)
# submit.reset_index(drop=True, inplace=True)
# test_data = []
# for i, row in submit.iterrows():
#     test_data.append(model.infer_vector(word_tokenize('. '.join(sel_data[(sel_data['sn']==row['sn'])&(sel_data['time']<=row['fault_time'])].tail(10)['msg']).lower())))
# test_feature = np.array(test_data)

In [ ]:
# test_label = rf.predict(test_feature)
# submit['label'] = test_label
# submit.to_csv('./preliminary_pred_df.csv', index=0)